In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import ReduceLROnPlateau
import visualkeras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras.layers import multiply  # Import the multiply function


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
top5_acc = TopKCategoricalAccuracy(k=5, name='top5_acc')
top10_acc = TopKCategoricalAccuracy(k=10, name='top10_acc')

In [3]:
root_path = r"C:\Users\Jarushen\Desktop\Masters Thesis\Images\Fynbos_Dataset_B"  # Your dataset path

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Adjust for ResNet50
    rotation_range=20,  # Increased rotation
    width_shift_range=0.2,  # Increased shift range
    height_shift_range=0.2,  # Increased shift range
    shear_range=0.2,  # Increased shear range
    zoom_range=0.2,  # Increased zoom range
    horizontal_flip=True,
    vertical_flip=True,  # New augmentation
    fill_mode='reflect',  # Changed from 'nearest' to 'reflect'
    brightness_range=[0.8,1.2],  # New augmentation for brightness
    channel_shift_range=20,  # New augmentation for color channels
    validation_split=0.2
)


# Load images from directory
train_generator = datagen.flow_from_directory(
    root_path,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    root_path,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 706 images belonging to 23 classes.
Found 163 images belonging to 23 classes.


In [4]:
from tensorflow.keras.layers import BatchNormalization

def squeeze_excite_block(input_tensor, ratio=16):
    # Get the number of channels (filters) in the input tensor
    channels = input_tensor.shape[-1]
    
    # Squeeze: Global average pooling
    se = GlobalAveragePooling2D()(input_tensor)
    se = Reshape((1, 1, channels))(se)
    
    # Excitation: Two fully connected layers with Batch Normalization
    se = Dense(channels // ratio, use_bias=False)(se)
    se = BatchNormalization()(se)
    se = Activation('relu')(se)
    
    se = Dense(channels, use_bias=False)(se)
    se = BatchNormalization()(se)
    se = Activation('sigmoid')(se)
    
    # Scale the input tensor by the SE block output
    x = Multiply()([input_tensor, se])
    return x

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Reshape, Multiply, Activation, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


# Load the base model, pre-trained on ImageNet
input_tensor = Input(shape=(299, 299, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Unfreeze the last 30 layers of the base model
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Add the SE block to the base model's output
x = base_model.output
x = squeeze_excite_block(x)

# Continue with the rest of the network
x = GlobalAveragePooling2D()(x)
x = Dense(512, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.3)(x)

predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)






# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with a reduced initial learning rate
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'top_k_categorical_accuracy']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001, verbose=1)

# Train the model with callbacks
history = model.fit(
    train_generator,
    epochs=20,  # Adjust as needed
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)




C:\ProgramData\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/20
23/23 [==============================] - 2924s 127s/step - loss: 2.8780 - acc: 0.1969 - top_k_categorical_accuracy: 0.4958 - val_loss: 2.6999 - val_acc: 0.1288 - val_top_k_categorical_accuracy: 0.4724
Epoch 2/20
23/23 [==============================] - 2890s 126s/step - loss: 1.6406 - acc: 0.5666 - top_k_categorical_accuracy: 0.8612 - val_loss: 2.3390 - val_acc: 0.3374 - val_top_k_categorical_accuracy: 0.6748
Epoch 3/20
23/23 [==============================] - 3377s 147s/step - loss: 1.2119 - acc: 0.7096 - top_k_categorical_accuracy: 0.9263 - val_loss: 2.0161 - val_acc: 0.4601 - val_top_k_categorical_accuracy: 0.7975
Epoch 4/20
14/23 [=================>............] - ETA: 20:08 - loss: 0.8838 - acc: 0.8134 - top_k_categorical_accuracy: 0.9737

# Fynbos Leaf Optical Recognition: Optimised Model B Workflow

This notebook consolidates all logic for:
- Training an optimised ResNet50-based model (Model B) with advanced augmentations and SE block
- Evaluating and saving the model
- Predicting on new images
- Generating SHAP explainability plots for predictions

Update paths as needed for your environment (local or Google Drive).

In [ ]:
# --- SHAP Explainability for Model B ---
import shap
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import os

# Function to load and preprocess a single image for prediction and SHAP

def load_preprocess_image(img_path, target_size=(299, 299)):
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
    return img_array

# Example: Predict and explain a new image
img_path = 'PATH/TO/NEW/IMAGE.jpg'  # Update this path
preprocessed_image = load_preprocess_image(img_path)
predictions = model.predict(preprocessed_image)
predicted_class = np.argmax(predictions, axis=1)[0]
class_names = list(train_generator.class_indices.keys())
print(f"Predicted class: {class_names[predicted_class]} ({100*predictions[0][predicted_class]:.2f}%)")

# SHAP DeepExplainer setup (use a small background dataset for efficiency)
background = next(train_generator)[0][:10]  # 10 random images from training set
explainer = shap.DeepExplainer(model, background)
shap_values = explainer.shap_values(preprocessed_image)

# Plot SHAP values for the predicted class
shap.image_plot(shap_values, preprocessed_image, show=False)
plt.title(f"SHAP Explanation for {class_names[predicted_class]}")
plt.show()